# Basics

## Taxonomies basic information

In [ ]:
from pytaxonomies import Taxonomies

taxonomies = Taxonomies()
print(f'Version: {taxonomies.version}')
print(f'License: {taxonomies.license}')
print(f'Description: {taxonomies.description}')
print(f'Number of taxonomies: {len(taxonomies)}')
print(f'Names: {list(taxonomies.keys())}')

## Get information on a specific taxonomy

In [ ]:
# Pick whichever taxonomy name
namespace = 'honeypot-basic'

### Standard information on the taxonomy

In [ ]:
taxonomy = taxonomies.get(namespace)
for feature in ('name', 'version', 'description'):
    print(f'{feature}: {getattr(taxonomy, feature)}')
print()
print(f'Number of predicates: {len(taxonomy)}')
print(f'Number of entries: {taxonomy.amount_entries()}')

### Overview of the predicates

In [ ]:
print(f'predicates: {list(taxonomy.keys())}\n')

for predicate, values in taxonomy.predicates.items():
    if len(values.keys()) == 0:
        print(predicate)
    else:
        print(f'{predicate}:')
        for value in values:
            print(f' - {value}')

### Detailed information on the predicates and their values

In [ ]:
# Pick wichever of the predicates displayed above
predicate_name = 'data-capture'
predicate = taxonomy.get(predicate_name)

In [ ]:
# In the case of TLP & PAP, there are only predicates
if len(predicate.keys()) == 0:
    for feature in ('predicate', 'expanded', 'description'):
        print(f'{feature}: {getattr(predicate, feature)}')
else:
    for value in predicate.keys():
        print(f'{value}:')
        for feature in ('value', 'expanded', 'description'):
            print(f' - {feature}: {getattr(predicate.get(value), feature)}')

#### And the easiest way

In [ ]:
# It will fail if there are only predicates and no values

value = 'network-capture'
print(f'value: {predicate.get(value).value}')
print(f'expanded value: {predicate.get(value).expanded}')
print(f'description: {predicate.get(value).description}')
print()

# or directly
print(taxonomies.get(namespace).get(predicate_name).get(value).value)

### Display the tags

In [ ]:
print(taxonomy.machinetags())
print(f'\nAnd with the expanded values:\n{taxonomy.machinetags_expanded()}')

# Create a new taxonomy

## Declare a new Taxonomy

In [ ]:
from pytaxonomies import Taxonomy

new_taxonomy = Taxonomy()

new_taxonomy.name = "dark-web"
new_taxonomy.description = (
    "Criminal motivation and content detection the dark web: A categorisation model for law enforcement. "
    "ref: Janis Dalins, Campbell Wilson, Mark Carman. "
    "Taxonomy updated by MISP Project and extended by the JRC (Joint Research Centre) of the European Commission."
)
new_taxonomy.version = 20240209
new_taxonomy.expanded = "Dark Web"

print(new_taxonomy.to_json())

## Load the predicates data

In [ ]:
from pathlib import Path

with open(Path().resolve() / 'test_taxonomy_predicates.csv') as f:
    predicate_lines = f.readlines()

print(lines[0])

## Declare the predicates

We declare the predicates and store them so we can add entries later.

In [ ]:
from pytaxonomies import Predicate

predicates = {}

for line in predicate_lines[1:]:
    new_predicate = Predicate()
    value, expanded, description = line.strip('\n').split('|')
    new_predicate.predicate = value
    new_predicate.expanded = expanded
    new_predicate.description = description
    print(new_predicate.to_json())
    predicates[value] = new_predicate

## Load the entries data

In [ ]:
with open(Path().resolve() / 'test_taxonomy_entries.csv') as f:
    entry_lines = f.readlines()

# Show the header
print(lines[0])

## Declare the entries

We declare the entries and add them to the related predicate

In [ ]:
from pytaxonomies import Entry

for line in entry_lines[1:]:
    predicate, value, expanded, description = line.strip('\n').split('|')
    entry = Entry()
    entry.value = value
    entry.expanded = expanded
    entry.description = description
    predicates[predicate].entries[value] = entry


## Add the predicates to the new taxonomy

In [ ]:
new_taxonomy.predicates = {name: predicate for name, predicate in predicates.items()}

In [ ]:
print(new_taxonomy.machinetags())

## Add the new taxonomy in the repository

In [ ]:
import json
from pathlib import Path

taxonomies_path = Path().resolve().parent / 'PyTaxonomies' / 'pytaxonomies' / 'data' / 'misp-taxonomies'

with open(taxonomies_path / 'MANIFEST.json', 'rt', encoding='utf-8') as m:
    manifest = json.load(m)
    
# We check if the taxonomy already exists.
is_new = True
for taxonomy in manifest['taxonomies']:
    if taxonomy['name'] == new_taxonomy.name:
        # The taxonomy already exists, we update it
        taxonomy['version'] += 1
        taxonomy['description'] = new_taxonomy.description
        is_new = False
        
if is_new:
    # It is a new taxonomy, we add it in the manifest
    manifest['taxonomies'].append(
        {
            'version': new_taxonomy.version,
            'name': new_taxonomy.name,
            'description': new_taxonomy.description
        }
    )
    
with open(taxonomies_path / 'MANIFEST.json', 'wt', encoding='utf-8') as f:
    f.write(json.dumps(manifest, indent=2, ensure_ascii=False))
    
if not (taxonomies_path / new_taxonomy.name).exists():
    (taxonomies_path / new_taxonomy.name).mkdir()
    
with open(taxonomies_path / new_taxonomy.name / 'machinetag.json', 'wt', encoding='utf-8') as f:
    json.dump(new_taxonomy.to_dict(), f, indent=2, ensure_ascii=False)

# Edit a taxonomy

In [ ]:
from pytaxonomies import Taxonomies

taxonomies = Taxonomies()

edited_taxonomy = taxonomies['dark-web']

edited_taxonomy.version = 5

## Save the edited taxonomy

In [ ]:
import json
from pathlib import Path

taxonomies_path = Path().resolve().parent / 'PyTaxonomies' / 'pytaxonomies' / 'data' / 'misp-taxonomies'

with open(taxonomies_path / 'MANIFEST.json', 'rt', encoding='utf-8') as m:
    manifest = json.load(m)
    
# We check if the taxonomy already exists.
exists = False
for taxonomy in manifest['taxonomies']:
    if taxonomy['name'] == edited_taxonomy.name:
        taxonomy['version'] = edited_taxonomy.version
        taxonomy['description'] = edited_taxonomy.description
        exists = True
        
if not exists:
    raise Exception(f'The {edited_taxonomy.name} Taxonomy does not exist in the manifest.')
    
with open(taxonomies_path / 'MANIFEST.json', 'w', encoding='utf-8') as f:
    json.dump(manifest, f, indent=2, ensure_ascii=False)
    
with open(taxonomies_path / edited_taxonomy.name / 'machinetag.json', 'w', encoding='utf-8') as f:
    json.dump(edited_taxonomy.to_dict(), f, indent=2, ensure_ascii=False)